In [1]:
import findspark
import streamlit
findspark.init()

In [1]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello")
df.show()

/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/11 08:10:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+
|hello|
+-----+
|spark|
+-----+



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = spark.read.csv("BrokenDataset.csv")
df.show()

+------+--------------+--------------------+------------+--------------------+----------+-------------+-------------+
|   _c0|           _c1|                 _c2|         _c3|                 _c4|       _c5|          _c6|          _c7|
+------+--------------+--------------------+------------+--------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of...|       lunch|test preparation ...|math score|reading score|writing score|
|female|       group B|   bachelor's degree|    standard|                none|        72|           72|           74|
|female|       group C|        some college|    standard|           completed|        69|           90|           88|
|female|       group B|     master's degree|    standard|                none|        90|           95|           93|
|  male|       group A|  associate's degree|free/reduced|                none|        47|           57|           44|
|  male|       group C|        some college|    standard

In [12]:
from functools import reduce

oldColumns = df.schema.names
newColumns = ['gender', 'race/ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course', 'math_score', 'reading_score', 'writing_score']
df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df)
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental_level_of_education|       lunch|test_preparation_course|math_score|reading_score|writing_score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|       parental level of...|       lunch|   test preparation ...|math score|reading score|writing score|
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|     

# ETL

In [19]:
for col in df.columns:
    count = df.where(df[col].isNull()).count()
    print(f"There are {count} null values in {col}")


There are 15 null values in gender
There are 0 null values in race/ethnicity
There are 9 null values in parental_level_of_education
There are 8 null values in lunch
There are 11 null values in test_preparation_course
There are 1 null values in math_score
There are 6 null values in reading_score
There are 1 null values in writing_score


In [21]:
df.count()

1001

## remove missing rows

In [27]:
df = df.dropna()
print(f"Removed {1001 - df.count()} Rows")

Removed 40 Rows


### Drop Duplicate Data

In [28]:
oldCount = df.count()
df = df.dropDuplicates()
print(f"Removed {oldCount - df.count()} Rows")

Removed 0 Rows


In [29]:
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental_level_of_education|       lunch|test_preparation_course|math_score|reading_score|writing_score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group B|               some college|free/reduced|                   none|        40|           43|           39|
|  male|       group B|           some high school|    standard|              completed|        61|           56|           56|
|female|       group C|           some high school|free/reduced|                   none|        65|           86|           80|
|female|       group C|         associate's degree|free/reduced|                   none|        60|           75|           74|
|female|       group C|               some college|    standard|                   none|        54|     

### dropping columns

In [44]:
df = df.where(df.gender != 'gender')

In [30]:
cols = ['test_preparation_course']
df = df.drop(*cols)
df.show()

+------+--------------+---------------------------+------------+----------+-------------+-------------+
|gender|race/ethnicity|parental_level_of_education|       lunch|math_score|reading_score|writing_score|
+------+--------------+---------------------------+------------+----------+-------------+-------------+
|  male|       group B|               some college|free/reduced|        40|           43|           39|
|  male|       group B|           some high school|    standard|        61|           56|           56|
|female|       group C|           some high school|free/reduced|        65|           86|           80|
|female|       group C|         associate's degree|free/reduced|        60|           75|           74|
|female|       group C|               some college|    standard|        54|           48|           52|
|  male|       group D|                high school|    standard|        89|           87|           79|
|female|       group C|                high school|    standard|

## Computations on Column Data

In [35]:
from math import floor
df.withColumn("avg_score", (df.math_score + df.reading_score + df.writing_score)/3)\
  .show()

+------+--------------+---------------------------+------------+----------+-------------+-------------+------------------+
|gender|race/ethnicity|parental_level_of_education|       lunch|math_score|reading_score|writing_score|         avg_score|
+------+--------------+---------------------------+------------+----------+-------------+-------------+------------------+
|  male|       group B|               some college|free/reduced|        40|           43|           39|40.666666666666664|
|  male|       group B|           some high school|    standard|        61|           56|           56|57.666666666666664|
|female|       group C|           some high school|free/reduced|        65|           86|           80|              77.0|
|female|       group C|         associate's degree|free/reduced|        60|           75|           74| 69.66666666666667|
|female|       group C|               some college|    standard|        54|           48|           52|51.333333333333336|
|  male|       g

## Count Categories

In [53]:
df = df.filter(df.gender.like("%male"))

for col in ['gender','race/ethnicity','parental_level_of_education','lunch']:
    print(f"{col} Categories: {df.groupBy(col).count().count()}")


df.groupBy('gender').count().show()

gender Categories: 2
race/ethnicity Categories: 5
parental_level_of_education Categories: 8
lunch Categories: 2
+------+-----+
|gender|count|
+------+-----+
|female|  499|
|  male|  460|
+------+-----+



## Encoding Values

In [56]:
from pyspark.ml.feature import StringIndexer

textCols = ['gender','race/ethnicity','parental_level_of_education','lunch']

indexer = StringIndexer(inputCols=textCols,
                        outputCols=list(map(lambda text: f"{text}_encoded", textCols))
                        )
encoded = indexer.fit(df).transform(df)
encoded.show()

+------+--------------+---------------------------+------------+----------+-------------+-------------+--------------+----------------------+-----------------------------------+-------------+
|gender|race/ethnicity|parental_level_of_education|       lunch|math_score|reading_score|writing_score|gender_encoded|race/ethnicity_encoded|parental_level_of_education_encoded|lunch_encoded|
+------+--------------+---------------------------+------------+----------+-------------+-------------+--------------+----------------------+-----------------------------------+-------------+
|  male|       group B|               some college|free/reduced|        40|           43|           39|           1.0|                   2.0|                                0.0|          1.0|
|  male|       group B|           some high school|    standard|        61|           56|           56|           1.0|                   2.0|                                3.0|          0.0|
|female|       group C|           some h

In [ ]:
# exprs = [col(column).alias(column.replace(' ', '_')) for column in df.columns]
# df.select(*exprs).show()

In [57]:
from pyspark.sql.functions import split
df_new = df.withColumn('race group', split(df['race/ethnicity'], ' ').getItem(1))
df_new.show()

+------+--------------+---------------------------+------------+----------+-------------+-------------+----------+
|gender|race/ethnicity|parental_level_of_education|       lunch|math_score|reading_score|writing_score|race group|
+------+--------------+---------------------------+------------+----------+-------------+-------------+----------+
|  male|       group B|               some college|free/reduced|        40|           43|           39|         B|
|  male|       group B|           some high school|    standard|        61|           56|           56|         B|
|female|       group C|           some high school|free/reduced|        65|           86|           80|         C|
|female|       group C|         associate's degree|free/reduced|        60|           75|           74|         C|
|female|       group C|               some college|    standard|        54|           48|           52|         C|
|  male|       group D|                high school|    standard|        89|     

# Filling Values

In [68]:
df_again = spark.read.csv("BrokenDataset.csv")
df_again = df_again.filter(df_again._c0 != 'gender')
df_again = df_again.fillna({"_c4": "none", "_c5": 0})
df_again.show()

+------+-------+------------------+------------+---------+---+---+---+
|   _c0|    _c1|               _c2|         _c3|      _c4|_c5|_c6|_c7|
+------+-------+------------------+------------+---------+---+---+---+
|female|group B| bachelor's degree|    standard|     none| 72| 72| 74|
|female|group C|      some college|    standard|completed| 69| 90| 88|
|female|group B|   master's degree|    standard|     none| 90| 95| 93|
|  male|group A|associate's degree|free/reduced|     none| 47| 57| 44|
|  male|group C|      some college|    standard|     none| 76| 78| 75|
|female|group B|associate's degree|    standard|     none| 71| 83| 78|
|female|group B|      some college|    standard|completed| 88| 95| 92|
|  male|group B|      some college|free/reduced|     none| 40| 43| 39|
|  male|group D|       high school|free/reduced|completed| 64| 64| 67|
|female|group B|       high school|free/reduced|     none| 38| 60| 50|
|  male|group C|associate's degree|    standard|     none| 58| 54| 52|
|  mal

# Graphing 

In [70]:
import matplotlib.pyplot as plt

toy_df = spark.read.csv("toy_dataset.csv",inferSchema=True, header=True)
toy_df.show()



+------+------+------+---+-------+-------+
|Number|  City|Gender|Age| Income|Illness|
+------+------+------+---+-------+-------+
|     1|Dallas|  Male| 41|40367.0|     No|
|     2|Dallas|  Male| 54|45084.0|     No|
|     3|Dallas|  Male| 42|52483.0|     No|
|     4|Dallas|  Male| 40|40941.0|     No|
|     5|Dallas|  Male| 46|50289.0|     No|
|     6|Dallas|Female| 36|50786.0|     No|
|     7|Dallas|Female| 32|33155.0|     No|
|     8|Dallas|  Male| 39|30914.0|     No|
|     9|Dallas|  Male| 51|68667.0|     No|
|    10|Dallas|Female| 30|50082.0|     No|
|    11|Dallas|Female| 48|41524.0|    Yes|
|    12|Dallas|  Male| 47|54777.0|     No|
|    13|Dallas|  Male| 46|62749.0|     No|
|    14|Dallas|Female| 42|50894.0|     No|
|    15|Dallas|Female| 61|38429.0|     No|
|    16|Dallas|  Male| 43|34074.0|     No|
|    17|Dallas|  Male| 27|50398.0|     No|
|    18|Dallas|  Male| 38|46373.0|    Yes|
|    19|Dallas|  Male| 47|51137.0|     No|
|    20|Dallas|Female| 35|23688.0|     No|
+------+---